In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
import random
import os
import re
import itertools

sns.set()
%matplotlib inline

In [11]:
df = pd.read_csv("Data.csv")
df["Doc. Date"] = pd.to_datetime(df["Doc. Date"])
df.head()

,Unnamed: 0,Doc. Date,Material,Order qty,Ship-to nu,PCS delivered,HL delivered,delivery_days,delivery_flag,lead_time_creation_vs_rdd_flag,...,Container Size,Variétés,Segment LE,Latitude,Longitude,Degre Alc,TTC,Brut + TE,Net + TE - Hors majoration de rompu de palette,DA
0,0,2021-01-11,10946,5.0,29606863,350.0,21.0,7,1,1,...,"6,000 L",Blonde,PREMIUM,50.6455,2.9619,6.6,18.137448,15.582,15.114540,0.0
1,1,2021-01-11,19898,2.0,29606863,140.0,8.4,7,1,1,...,NaN,NaN,NaN,50.6455,2.9619,5.0,17.396467,14.688,14.497056,0.0
2,2,2021-01-11,80176,4.0,29606863,280.0,16.8,7,1,1,...,NaN,NaN,NaN,50.6455,2.9619,0.0,12.279859,10.368,10.233216,0.0
3,3,2021-01-11,66989,1.0,29606863,70.0,4.2,7,1,1,...,"6,000 L",-,SUPER PREMIUM,50.6455,2.9619,8.0,21.996000,18.330,18.330000,0.0
4,4,2021-01-08,11766,70.0,29378784,70.0,4.2,6,1,1,...,NaN,NaN,NaN,50.6978,2.2423,5.2,17.359200,14.466,14.466000,0.0


In [12]:
materials = df["Material"].dropna().unique()
wholesalers = df["Ship-to nu"].dropna().unique()

In [13]:
df_datewise = pd.DataFrame(columns=["Wholesaler", "Date", "HLs", "Materials"])
date_range = pd.date_range(start="2019-08-01", end="2021-02-22")
for wholesaler in tqdm(wholesalers):
    df1 = df[df["Ship-to nu"] == wholesaler]
    for date in date_range:
        df2 = df1[df1["Doc. Date"] == date]
        if(len(df2) == 0):
            continue
        df2 = df2.groupby("Material").agg({"HL delivered": np.sum})
        HLs = list(df2["HL delivered"])
        materials = list(df2.index)
        df_datewise.loc[len(df_datewise.index)] = [wholesaler, date, HLs, materials]

100%|████████████████████████████████████████████████████████████████████████████████| 179/179 [01:12<00:00,  2.45it/s]


In [19]:
df_datewise

,Wholesaler,Date,HLs,Materials
0,29606863,2019-08-12,"[1.584, 16.8, 4.752, 28.8, 8.4, 8.4, 4.0, 4.2,...","[3338, 10946, 10957, 11574, 19898, 20614, 3796..."
1,29606863,2019-08-14,"[14.4, 6.4, 8.0, 16.8, 6.4, 13.5, 4.2, 32.4, 9...","[3372, 6013, 9974, 10946, 10965, 16550, 20614,..."
2,29606863,2019-08-19,"[33.264, 4.752, 37.8, 25.2, 4.752]","[3337, 10957, 29777, 57005, 59873]"
3,29606863,2019-08-21,"[2.376, 28.8, 57.6, 4.2, 21.6, 13.5, 4.2, 12.6...","[3338, 3372, 3410, 10947, 11574, 16550, 19898,..."
4,29606863,2019-08-27,"[43.2, 8.0, 1.2, 16.8, 9.6, 9.0, 1.2, 12.6, 4....","[3372, 9974, 10611, 10946, 10965, 16519, 48537..."
...,...,...,...,...
5816,29801344,2020-07-09,"[5.8788, 20.448, 5.7024]","[63445, 77313, 77589]"
5817,29801344,2020-07-28,"[6.24, 12.48, 6.24, 11.4048]","[62474, 63285, 72414, 76623]"
5818,29801344,2020-07-30,[11.4048],[77589]
5819,29801344,2020-08-12,"[5.8788, 13.8024, 11.4048]","[63445, 77313, 77589]"


In [20]:
df_datewise.to_csv("WholesalerDatewise.csv", index=False)

In [13]:
df_rating = pd.DataFrame(columns=["Material", "Wholesaler", "HLs", "Dates"])
for material, wholesaler in tqdm(itertools.product(materials, wholesalers)):
    df1 = df[df["Material"] == material]
    df2 = df1[df1["Ship-to nu"] == wholesaler]
    if len(df2) == 0:
        continue
    df2 = df2.sort_values(by=["Doc. Date"])
    df2 = df2.groupby("Doc. Date").agg({"HL delivered": np.sum})
    # ttc_s = df2["TTC"].fillna(average_ttc_subbrand[subbrand])
    HLs = list(df2["HL delivered"])
    dates = list(df2.index)
    df_rating.loc[len(df_rating.index)] = [material, wholesaler, HLs, dates]

32041it [03:31, 151.82it/s]


In [16]:
df_rating.head(60)

,Material,Wholesaler,HLs,Dates
0,10946,29606863,"[16.8, 16.8, 16.8, 79.8, 12.6, 21.0, 16.8, 16....","[2019-08-12 00:00:00, 2019-08-14 00:00:00, 201..."
1,10946,29378784,"[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 8.4, 4.2, 4.2, ...","[2019-08-27 00:00:00, 2019-09-13 00:00:00, 201..."
2,10946,29379853,"[42.0, 50.4, 33.6, 21.0, 109.2, 42.0, 71.4, 29...","[2019-08-01 00:00:00, 2019-08-05 00:00:00, 201..."
3,10946,29379855,"[25.2, 4.2, 8.4, 8.4, 8.4, 8.4, 16.8, 8.4, 8.4...","[2019-08-05 00:00:00, 2019-08-12 00:00:00, 201..."
4,10946,29379861,"[46.2, 8.4, 71.4, 4.2, 67.2, 21.0, 109.2, 25.2...","[2019-08-12 00:00:00, 2019-08-22 00:00:00, 201..."
5,10946,29379635,"[12.6, 12.6, 67.2, 4.2, 8.4, 12.6, 16.8, 8.4, ...","[2019-08-05 00:00:00, 2019-08-30 00:00:00, 201..."
6,10946,29379829,"[25.2, 8.4, 29.4, 67.2, 25.2, 16.8, 16.8, 16.8...","[2019-08-09 00:00:00, 2019-08-13 00:00:00, 201..."
7,10946,29378785,"[109.2, 109.2, 92.4, 84.0, 25.2, 121.8, 180.60...","[2019-08-27 00:00:00, 2019-09-09 00:00:00, 201..."
8,10946,29379661,"[4.2, 63.0, 12.6, 63.0, 8.4, 4.2, 16.8, 4.2, 2...","[2019-08-01 00:00:00, 2019-08-12 00:00:00, 201..."
9,10946,29635311,"[16.8, 58.8, 117.60000000000001, 29.4, 33.6, 1...","[2019-08-05 00:00:00, 2019-08-13 00:00:00, 201..."


In [10]:
df_rating.to_csv("HLPerDay.csv")

In [12]:
data = pd.read_csv("HLPerDay.csv")

In [13]:
data["HLs"] = data["HLs"].apply(eval)

In [14]:
data

,Unnamed: 0,Material,Wholesaler,HLs,Dates
0,0,10946,29606863,"[16.8, 16.8, 16.8, 79.8, 12.6, 21.0, 16.8, 16....","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
1,1,10946,29378784,"[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 8.4, 4.2, 4.2, ...","[Timestamp('2019-08-27 00:00:00'), Timestamp('..."
2,2,10946,29379853,"[42.0, 50.4, 33.6, 21.0, 109.2, 42.0, 71.4, 29...","[Timestamp('2019-08-01 00:00:00'), Timestamp('..."
3,3,10946,29379855,"[25.2, 4.2, 8.4, 8.4, 8.4, 8.4, 16.8, 8.4, 8.4...","[Timestamp('2019-08-05 00:00:00'), Timestamp('..."
4,4,10946,29379861,"[46.2, 8.4, 71.4, 4.2, 67.2, 21.0, 109.2, 25.2...","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
...,...,...,...,...,...
8323,8323,15757,29595514,[0.0],[Timestamp('2019-12-23 00:00:00')]
8324,8324,77393,29563817,[19.2],[Timestamp('2019-12-24 00:00:00')]
8325,8325,65643,29446255,[142.8],[Timestamp('2020-05-18 00:00:00')]
8326,8326,65643,29750335,[0.6],[Timestamp('2020-05-14 00:00:00')]


In [15]:
# data = data[data["HLs"].apply(len) > 1]
print(f"length = {len(data)}")
data = data.reset_index()
data = data.drop(columns=["index", "Unnamed: 0"])
data

length = 8328


,Material,Wholesaler,HLs,Dates
0,10946,29606863,"[16.8, 16.8, 16.8, 79.8, 12.6, 21.0, 16.8, 16....","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
1,10946,29378784,"[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 8.4, 4.2, 4.2, ...","[Timestamp('2019-08-27 00:00:00'), Timestamp('..."
2,10946,29379853,"[42.0, 50.4, 33.6, 21.0, 109.2, 42.0, 71.4, 29...","[Timestamp('2019-08-01 00:00:00'), Timestamp('..."
3,10946,29379855,"[25.2, 4.2, 8.4, 8.4, 8.4, 8.4, 16.8, 8.4, 8.4...","[Timestamp('2019-08-05 00:00:00'), Timestamp('..."
4,10946,29379861,"[46.2, 8.4, 71.4, 4.2, 67.2, 21.0, 109.2, 25.2...","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
...,...,...,...,...
8323,15757,29595514,[0.0],[Timestamp('2019-12-23 00:00:00')]
8324,77393,29563817,[19.2],[Timestamp('2019-12-24 00:00:00')]
8325,65643,29446255,[142.8],[Timestamp('2020-05-18 00:00:00')]
8326,65643,29750335,[0.6],[Timestamp('2020-05-14 00:00:00')]


In [16]:
for i in data.index:
    data.at[i, "HLs"] = [min(x, 200) for x in data.loc[i]["HLs"]]
data

,Material,Wholesaler,HLs,Dates
0,10946,29606863,"[16.8, 16.8, 16.8, 79.8, 12.6, 21.0, 16.8, 16....","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
1,10946,29378784,"[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 8.4, 4.2, 4.2, ...","[Timestamp('2019-08-27 00:00:00'), Timestamp('..."
2,10946,29379853,"[42.0, 50.4, 33.6, 21.0, 109.2, 42.0, 71.4, 29...","[Timestamp('2019-08-01 00:00:00'), Timestamp('..."
3,10946,29379855,"[25.2, 4.2, 8.4, 8.4, 8.4, 8.4, 16.8, 8.4, 8.4...","[Timestamp('2019-08-05 00:00:00'), Timestamp('..."
4,10946,29379861,"[46.2, 8.4, 71.4, 4.2, 67.2, 21.0, 109.2, 25.2...","[Timestamp('2019-08-12 00:00:00'), Timestamp('..."
...,...,...,...,...
8323,15757,29595514,[0.0],[Timestamp('2019-12-23 00:00:00')]
8324,77393,29563817,[19.2],[Timestamp('2019-12-24 00:00:00')]
8325,65643,29446255,[142.8],[Timestamp('2020-05-18 00:00:00')]
8326,65643,29750335,[0.6],[Timestamp('2020-05-14 00:00:00')]


In [17]:
def convert_dates(dates):
    if len(dates) < 4:
        return []
    return [pd.to_datetime(re.search('\'(.*)\'', s)[1]) for s in dates.split(", ")]
data["Dates"] = data["Dates"].apply(convert_dates)

In [18]:
def date_difference(dates):
    diff = []
    for i in range(1, len(dates)):
        diff.append((dates[i] - dates[i-1]).days)
    diff.append(1)
    return diff
data["Date Difference"] = data["Dates"].apply(date_difference)
data

,Material,Wholesaler,HLs,Dates,Date Difference
0,10946,29606863,"[16.8, 16.8, 16.8, 79.8, 12.6, 21.0, 16.8, 16....","[2019-08-12 00:00:00, 2019-08-14 00:00:00, 201...","[2, 13, 3, 47, 7, 12, 16, 2, 10, 17, 29, 12, 2..."
1,10946,29378784,"[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 8.4, 4.2, 4.2, ...","[2019-08-27 00:00:00, 2019-09-13 00:00:00, 201...","[17, 20, 22, 27, 15, 17, 37, 22, 162, 6, 40, 1..."
2,10946,29379853,"[42.0, 50.4, 33.6, 21.0, 109.2, 42.0, 71.4, 29...","[2019-08-01 00:00:00, 2019-08-05 00:00:00, 201...","[4, 16, 20, 9, 49, 28, 61, 15, 48, 8, 1, 11, 5..."
3,10946,29379855,"[25.2, 4.2, 8.4, 8.4, 8.4, 8.4, 16.8, 8.4, 8.4...","[2019-08-05 00:00:00, 2019-08-12 00:00:00, 201...","[7, 2, 5, 8, 1, 7, 5, 11, 10, 7, 7, 7, 15, 13,..."
4,10946,29379861,"[46.2, 8.4, 71.4, 4.2, 67.2, 21.0, 109.2, 25.2...","[2019-08-12 00:00:00, 2019-08-22 00:00:00, 201...","[10, 7, 26, 16, 39, 3, 68, 22, 13, 97, 3, 26, ..."
...,...,...,...,...,...
8323,15757,29595514,[0.0],[2019-12-23 00:00:00],[1]
8324,77393,29563817,[19.2],[2019-12-24 00:00:00],[1]
8325,65643,29446255,[142.8],[2020-05-18 00:00:00],[1]
8326,65643,29750335,[0.6],[2020-05-14 00:00:00],[1]


In [19]:
data.to_csv("HLWholesalerMaterialPair.csv", index=False)